# Crypto 1 - Iterated Polynomials

> Took me a weekend on a laptop but finally I got an output!
> 
> - ariana
>
> Files: [main.py](main.py)

In this challenge, you start off with a polynomial `g`, and this polynomial is iteratively transformed via `g = f(g)` lots of times. Specifically, the number of iterations is the value of the flag treated as an int (as in `bytes_to_long()`).

Let's see what we have been given:
- $p = 2^{32} + 15$
- We work in the quotient ring $S = \mathbb{F}_p[X] / \langle a(X) \rangle$, where $a$ is some degree-16 polynomial we are given.
- The transformation function is given by
$$ f\left(\sum_{i=0}^{15} c_i X^{i} \right) = \sum_{i=0}^{15} c_i (X^2+1)^{i}.$$

It should be quite clear from this description that the transformation $f$ is linear. In that case, we can hope to represent it as a matrix $M$, so that each transformation is represented by $v_{i+1} = v_i M$, and this reduces the problem to finding the value of $n$ for which our final vector if $v_n = v_0 M^n$.

Let's get the values into sagemath first. Note that we have `y` instead of $X$ because sagemath likes to pretend they're different things.

In [1]:
p = 2^32+15
R = GF(p)[x]
a = x^16 + 2206327570*x^15 + 764008823*x^14 + 2624308288*x^13 + 584210452*x^12 + 2859245580*x^11 + 2161247258*x^10 + 2475621239*x^9 + 2679079*x^8 + 3262843933*x^7 + 3126742286*x^6 + 2840770970*x^5 + 2798946498*x^4 + 1178619281*x^3 + 124682568*x^2 + 150251198*x + 1469826103
S.<y> = R.quotient(a)
target = 4213081404*y^15 + 3296429821*y^14 + 4211675621*y^13 + 1980847685*y^12 + 1112259653*y^11 + 330484598*y^10 + 23881381*y^9 + 2112413024*y^8 + 2815876074*y^7 + 4197415602*y^6 + 3078141258*y^5 + 4163495398*y^4 + 4121679949*y^3 + 2775737979*y^2 + 1590517927*y + 1223073206
f = lambda g: sum(c*(y^2+1)^n for n,c in enumerate(g))

Ok, so what should our matrix and vector spaces look like? Each vector is a polynomial of degree at most 15, so this gives a natural isomorphism to $\mathbb{F}_p^{16}$. We can construct our matrix $M$ by transforming the basis elements. And for sanity check purposes, let's just see what the matrix looks like mod 1000.

In [2]:
M = matrix([vector(f(y^i)) for i in range(16)])
print(M.change_ring(ZZ) % 1000)

[  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  1   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  1   0   2   0   1   0   0   0   0   0   0   0   0   0   0   0]
[  1   0   3   0   3   0   1   0   0   0   0   0   0   0   0   0]
[  1   0   4   0   6   0   4   0   1   0   0   0   0   0   0   0]
[  1   0   5   0  10   0  10   0   5   0   1   0   0   0   0   0]
[  1   0   6   0  15   0  20   0  15   0   6   0   1   0   0   0]
[  1   0   7   0  21   0  35   0  35   0  21   0   7   0   1   0]
[209 113 751  30 841 341  81 378 302  72 109 731 887  23 496 741]
[392 977 353  39 549 516 934 956 491  90 155 852 709 362 107 665]
[221 562 930 200 722 845 221 414 726 233 923 952 260 375 340 140]
[980 391 990 175 741 119 957 475 500 927 361 933 309 465 629 823]
[ 53 670 872 332 398 162 484 538 921 884 312 158 638 600 997 987]
[564 704 807 191  40 628 678 429  59 993 557 601  51 857 127 494]
[225 726 730 289 672 382 657 632 639 762 235 512 716 865  52 206]
[413 277 1

At this point, the idea should be to find a fast way to evaluate $M^n$. The usual way to do this is via diagonalisation, i.e. finding an invertible matrix $P$ and a diagonal matrix $D$ such that $M = PDP^{-1}$. One thing to note is that $D$ contains the eigenvalues of $M$, which may not exist in $\mathbb{F}_p$ but rather some field extension of it. We could also work with the algebraic closure, but that's probably more trouble than it's worth. In any case, with some trial-and-error we find that all the eigenvalues are in $GF(p^{12})$.

In [3]:
D, P = M.diagonalization(GF(p^12))
assert M == P * D * P^-1

At this point we are ready to do some stuff. Let $w$ be the `target` vector, and $n$ be the `bytes_to_long()` value of the flag, i.e the number of iterations.

In [4]:
w = vector(target)
w

(1223073206, 1590517927, 2775737979, 4121679949, 4163495398, 3078141258, 4197415602, 2815876074, 2112413024, 23881381, 330484598, 1112259653, 1980847685, 4211675621, 3296429821, 4213081404)

We wish to solve the equation $w = v_0 M^n = v_0 P D^n P^{-1}$.

This is equivalent to solving $wP = (v_0 P) D^n$. Since $wP$ and $v_0 P$ are vectors, and $D$ is a diagonal matrix, this means that each diagonal entry of $D$ only acts on a single coordinate of the vector.

To demonstrate this by way of example, the equation could look something like this:
$$ \begin{pmatrix}
12 & 18 & 16
\end{pmatrix}
=
\begin{pmatrix}
3 & 2 & 1
\end{pmatrix}
\begin{pmatrix}
2 & 0 & 0 \\
0 & 3 & 0 \\
0 & 0 & 4
\end{pmatrix} ^ 2.$$

In other words, we can break this up into a bunch of independent discrete logs.

In [5]:
dlog_targets = [a/b for a,b in zip(w*P, vector(y)*P)]
dlogs = []
mods = []
for i, t in enumerate(dlog_targets):
    d = D[i][i]
    if d == t == 1:
        dlog, mod = 0, 1
    else:
        dlog, mod = t.log(d), d.multiplicative_order()
    dlogs.append(dlog)
    mods.append(mod)
    print(i, dlog, mod)

0 0 1
1 39659895313370669262782774899 79228163344367823809576701230
2 30244136201289835 76861434177327378
3 30244136201289835 76861434177327378
4 68832491842489549589549111574014464981553494769485937929 261545911121742886336251906983649969595675641752826538140
5 39659895313370669262782774899 79228163344367823809576701230
6 160904653911627958279069599163942529 18904576204146013290129967807464358880
7 68832491842489549589549111574014464981553494769485937929 261545911121742886336251906983649969595675641752826538140
8 68832491842489549589549111574014464981553494769485937929 261545911121742886336251906983649969595675641752826538140
9 160904653911627958279069599163942529 18904576204146013290129967807464358880
10 39659895313370669262782774899 79228163344367823809576701230
11 160904653911627958279069599163942529 18904576204146013290129967807464358880
12 68832491842489549589549111574014464981553494769485937929 261545911121742886336251906983649969595675641752826538140
13 688324918424895495895491

To be fair, there is a lot of redundant computation we could have skipped, seeing as the same values pop up multiple times. I guess it's a good confirmation for us that the process is consistent.

That's basically it. Now we can CRT all the discrete logs together and get the flag!

In [6]:
from Crypto.Util.number import *
long_to_bytes(crt(dlogs, mods))

b'7h3_FunC710N_15_4c7U4Lly_l1N34r!'

The flag is `grey{7h3_FunC710N_15_4c7U4Lly_l1N34r!}`.